In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import json

In [225]:
# read the url and use beautiful soup to parse HTML response
response = requests.get('https://www.vogue.com/fashion-shows/fall-2018-ready-to-wear/versace#details')
soup = bs(response.text, 'html.parser')

# retrieve the designer and season details from <title> tag
vogue_designer_season = soup.title.text

# retrieve date published for the collection
for item in json_data:
    date = re.search(r'"datePublished":"(.+?)T', str(x))
    if date:
        datepublished = str(date.group(1))
        break

Versace Fall 2018 Ready-to-Wear Collection - Vogue


In [6]:
# the website uses json data structure and java script within multiple <script> tags for the layout
# we will extract all of them using beautiful soup
json_data = soup.findAll(lambda tag: tag.name == 'script')

In [21]:
# from multiple <script> tags only one contains the required json data which we need to parse to get image urls. 
# we will use regex to identify the correct <script>


# Method - 1) This is one of doing it but there is still the <script> tags in it

for i in json_data:
    testing = re.search(r'react:', str(i))
    if testing:
        print(str(i))
        break

<script>
        window.__VOG__ = window.__VOG__ || {};
        window.__VOG__.initialState = { react: {"context":{"dispatcher":{"stores":{"ApplicationStore":{"channel":"runway","config":{"env":"production","siteConfig":{"allowedDomains":"www.vogue.com"}},"contentType":"runway_review","meta":{"canonical":"https://www.vogue.com/fashion-shows/fall-2018-ready-to-wear/versace","id":"","title":"","description":"","ogTitle":"","ogDescription":"","twitterDescription":"","socialImage":"https://www.vogue.com/wp-content/uploads/2016/02/09/voguerunway-share.jpg"},"pageName":"fashionShowReview","pubDate":"","primary":{"template":""},"query":{},"seasonSlug":"fall-2018-ready-to-wear","subChannel":"versace","subPageName":"","termSlug":"","tags":[],"template":"runway_review"},"MetaStore":{"canonical":"https://www.vogue.com/fashion-shows/fall-2018-ready-to-wear/versace","id":"","title":"","description":"","ogDescription":"","ogTitle":"","socialImage":"https://www.vogue.com/wp-content/uploads/2016/02/09

In [8]:
# Method - 2) This is another way of doing it and it does not have the <script> tags

# What I'm using is Regex (Regular Expression). 
# .group(1) works here because I'm using parenthesis in my regex. If I do group(0), it would be the whole value
    # including the <script> tags. 

my_json = []
for x in json_data:
    testing = re.search(r'(\{\s*react:.+?\}\s*\});\s*<\/script>', str(x))
    if testing:
        my_json.append(testing.group(1))
        break

In [35]:
my_json

['{ react: {"context":{"dispatcher":{"stores":{"ApplicationStore":{"channel":"runway","config":{"env":"production","siteConfig":{"allowedDomains":"www.vogue.com"}},"contentType":"runway_review","meta":{"canonical":"https://www.vogue.com/fashion-shows/fall-2018-ready-to-wear/versace","id":"","title":"","description":"","ogTitle":"","ogDescription":"","twitterDescription":"","socialImage":"https://www.vogue.com/wp-content/uploads/2016/02/09/voguerunway-share.jpg"},"pageName":"fashionShowReview","pubDate":"","primary":{"template":""},"query":{},"seasonSlug":"fall-2018-ready-to-wear","subChannel":"versace","subPageName":"","termSlug":"","tags":[],"template":"runway_review"},"MetaStore":{"canonical":"https://www.vogue.com/fashion-shows/fall-2018-ready-to-wear/versace","id":"","title":"","description":"","ogDescription":"","ogTitle":"","socialImage":"https://www.vogue.com/wp-content/uploads/2016/02/09/voguerunway-share.jpg","twitterDescription":""},"NewsletterStore":{"personalize":{"errorMes

In [59]:
# Every section in the website has a specific Client ID tagged. Fetch the Client ID tagged to all the images under "Detail" section
details_id = re.search(r'("detail":\s*\{\s*"__ref":\s*")(.+)=="\s*\},', str(my_json))
if details_id:
    id_det=details_id.group(2)
else:
    print("No Client ID for Detail retrieved")
    

In [192]:
# Fetch the number of edges available for the Client ID. 
# Based on the observed source code, if the 'slidecount'=n and n>100, the max number of edges that can be retrived from the page is 100. 
# If the 'slidecount'= n and n<100, the max number of edges that can be retrived from the page is n.
# We will use this variable to define our regex for url retrieval later
regex_edge_ct = r'"' + re.escape(details_id.group(2)) + r'==":\s*\{(.+?)"slidecount":\s*(.+?),"slidesV2'
slidecount = re.search(regex_edge_ct, str(my_json), re.IGNORECASE)
slidecount.group(2)

if int(slidecount.group(2)) >= 100:
    edgemax = 99
else:
    edgemax = int(slidecount.group(2))

print(edgemax)

99


In [193]:

# using regex and egemax variable we will extract the required portion of the json data which contains image urls
regex = r'"client:'+ re.escape(details_id.group(2)) + r'==:slidesV2\{\\\\"first\\\\\s*":100\}:edges:0":.*' + r'"client:'+ re.escape(details_id.group(2)) + r'==:slidesV2\{\\\\"first\\\\\s*":100\}:edges:' + re.escape(str(edgemax)) + r'".+?"client:' 
regex_ref1 = re.search(regex, str(my_json), re.IGNORECASE)
regex_ref1.group(0)


'"client:RmFzaGlvblNob3dHYWxsZXJ5OjVhOGI2YzdhYzkyYmQ1NzY4MTZkMmYyNg==:slidesV2{\\\\"first\\\\":100}:edges:0":{"__id":"client:RmFzaGlvblNob3dHYWxsZXJ5OjVhOGI2YzdhYzkyYmQ1NzY4MTZkMmYyNg==:slidesV2{\\\\"first\\\\":100}:edges:0","__typename":"GallerySlidesEdge","node":{"__ref":"U2xpZGU6NWE5MDgzM2M3NGRmYmYwYmQ1M2ZiNWE1"}},"U2xpZGU6NWE5MDgzM2M3NGRmYmYwYmQ1M2ZiNWE1":{"__id":"U2xpZGU6NWE5MDgzM2M3NGRmYmYwYmQ1M2ZiNWE1","__typename":"Slide","id":"U2xpZGU6NWE5MDgzM2M3NGRmYmYwYmQ1M2ZiNWE1","photosTout":{"__ref":"TWVkaWE6NWE5MDgzM2M3NGRmYmYwYmQ1M2ZiNWE1"}},"TWVkaWE6NWE5MDgzM2M3NGRmYmYwYmQ1M2ZiNWE1":{"__id":"TWVkaWE6NWE5MDgzM2M3NGRmYmYwYmQ1M2ZiNWE1","__typename":"Image","id":"TWVkaWE6NWE5MDgzM2M3NGRmYmYwYmQ1M2ZiNWE1","altText":"","resizedUrl":"https://assets.vogue.com/photos/5a90833c74dfbf0bd53fb5a5/master/pass/_ARC0010.jpg","url":"https://assets.vogue.com/photos/5a90833c74dfbf0bd53fb5a5/master/pass/_ARC0010.jpg"},"client:RmFzaGlvblNob3dHYWxsZXJ5OjVhOGI2YzdhYzkyYmQ1NzY4MTZkMmYyNg==:slidesV2{\\\\"firs

In [194]:
# fetch the URL of all images for the Client ID retrieved
pattern = re.compile(r'"resizedUrl":"(.+?)"')
images = pattern.findall(str(regex_ref1.group(0)))

In [195]:
images

['https://assets.vogue.com/photos/5a90833c74dfbf0bd53fb5a5/master/pass/_ARC0010.jpg',
 'https://assets.vogue.com/photos/5a908341dfd313614130fdd4/master/pass/_ARC0012.jpg',
 'https://assets.vogue.com/photos/5a90834269ad9276fa55a220/master/pass/_ARC0017.jpg',
 'https://assets.vogue.com/photos/5a9083401c7a764f97cd93ca/master/pass/_ARC0018.jpg',
 'https://assets.vogue.com/photos/5a90833d7cea34278b667c76/master/pass/_ARC0028.jpg',
 'https://assets.vogue.com/photos/5a9083442c9af205595386e2/master/pass/_ARC0040.jpg',
 'https://assets.vogue.com/photos/5a90834974dfbf0bd53fb5a9/master/pass/_ARC0050.jpg',
 'https://assets.vogue.com/photos/5a90834c1c7a764f97cd93cc/master/pass/_ARC0057.jpg',
 'https://assets.vogue.com/photos/5a90833e1a71770bab842c8d/master/pass/_ARC0069.jpg',
 'https://assets.vogue.com/photos/5a90834769ad9276fa55a222/master/pass/_ARC0071.jpg',
 'https://assets.vogue.com/photos/5a9083451415a85dc8348138/master/pass/_ARC0082.jpg',
 'https://assets.vogue.com/photos/5a90834aa8e2b84d5b0f

In [224]:
# create a dictionary of image url, collection, date published

data = pd.DataFrame({'imageurl':images})
data['collection'] = vogue_designer_season
data['datepublished'] = datepublished
print(data)

                                             imageurl  \
0   https://assets.vogue.com/photos/5a90833c74dfbf...   
1   https://assets.vogue.com/photos/5a908341dfd313...   
2   https://assets.vogue.com/photos/5a90834269ad92...   
3   https://assets.vogue.com/photos/5a9083401c7a76...   
4   https://assets.vogue.com/photos/5a90833d7cea34...   
5   https://assets.vogue.com/photos/5a9083442c9af2...   
6   https://assets.vogue.com/photos/5a90834974dfbf...   
7   https://assets.vogue.com/photos/5a90834c1c7a76...   
8   https://assets.vogue.com/photos/5a90833e1a7177...   
9   https://assets.vogue.com/photos/5a90834769ad92...   
10  https://assets.vogue.com/photos/5a9083451415a8...   
11  https://assets.vogue.com/photos/5a90834aa8e2b8...   
12  https://assets.vogue.com/photos/5a90834874dfbf...   
13  https://assets.vogue.com/photos/5a9083537cea34...   
14  https://assets.vogue.com/photos/5a9083521415a8...   
15  https://assets.vogue.com/photos/5a908351d9fbe8...   
16  https://assets.vogue.com/ph